In [13]:
## !pip install mlxtend
## !pip install onnxmltools
## !pip install onnxruntime
## !pip install torch torchvision
## !pip install skl2onnx

     -------------------------------------- 300.3/300.3 kB 1.2 MB/s eta 0:00:00
     ------------------------------------ 298.4/298.4 kB 878.6 kB/s eta 0:00:00
     -------------------------------------- 298.5/298.5 kB 1.1 MB/s eta 0:00:00


In [204]:
import numpy as np
import torch
import pandas as pd
import random
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [205]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

In [206]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [207]:
import onnxruntime as rt
import onnxmltools

from skl2onnx.common.data_types import FloatTensorType

In [309]:
## 0.001, 0.0001, 0.0003, 0.01, 0.03

batch_size    = 32
learning_rate = 0.001 ## 0.001
N_Epochs      = 100

epsilon = 0.0001

In [310]:
path_data = 'WeatherData.csv'

Weather_raw_data = pd.read_csv( path_data, delimiter="," )


In [311]:

Weather_raw_data

,Location,Temperature_C,Humidity_pct,Wind_Speed_kmh,Date_Time
0,Philadelphia,-8.63,54.07,26.37,2/26/2024 17:32
1,New York,-6.89,30.80,16.85,1/25/2024 19:04
2,New York,0.96,38.82,7.99,3/29/2024 5:20
3,New York,35.15,54.75,25.43,3/4/2024 13:47
4,Dallas,32.02,53.19,3.05,2/27/2024 21:07
...,...,...,...,...,...
495,Dallas,35.35,66.51,16.81,4/6/2024 3:57
496,Los Angeles,33.80,40.55,10.46,5/15/2024 4:28
497,Chicago,19.86,45.88,27.25,4/9/2024 4:35
498,Philadelphia,21.00,61.12,12.01,5/8/2024 5:39


In [312]:
headers_list = Weather_raw_data.columns.values.tolist()

headers_list


['Location', 'Temperature_C', 'Humidity_pct', 'Wind_Speed_kmh', 'Date_Time']

In [313]:
Weather_raw_data_np = Weather_raw_data.to_numpy()

In [314]:
Weather_raw_data_np

array([['Philadelphia', -8.63, 54.07, 26.37, '2/26/2024 17:32'],
       ['New York', -6.89, 30.8, 16.85, '1/25/2024 19:04'],
       ['New York', 0.96, 38.82, 7.99, '3/29/2024 5:20'],
       ...,
       ['Chicago', 19.86, 45.88, 27.25, '4/9/2024 4:35'],
       ['Philadelphia', 21.0, 61.12, 12.01, '5/8/2024 5:39'],
       ['Los Angeles', -3.04, 49.09, 28.33, '2/6/2024 18:00']],
      dtype=object)

In [315]:
Weather_raw_data_np.shape

(500, 5)

In [316]:
X = Weather_raw_data_np[:, 1:4]     
y = Weather_raw_data_np[:, 0]
time_column = Weather_raw_data_np[:, 4]

In [317]:

y

array(['Philadelphia', 'New York', 'New York', 'New York', 'Dallas',
       'Philadelphia', 'Chicago', 'New York', 'New York', 'Los Angeles',
       'New York', 'Dallas', 'Chicago', 'Philadelphia', 'Dallas',
       'Philadelphia', 'Chicago', 'New York', 'Dallas', 'New York',
       'Philadelphia', 'Chicago', 'Chicago', 'Philadelphia', 'New York',
       'New York', 'Philadelphia', 'Chicago', 'Dallas', 'Los Angeles',
       'Chicago', 'Los Angeles', 'New York', 'New York', 'New York',
       'Los Angeles', 'Dallas', 'Dallas', 'Chicago', 'Philadelphia',
       'Los Angeles', 'Dallas', 'Los Angeles', 'Los Angeles',
       'Philadelphia', 'Philadelphia', 'Chicago', 'Philadelphia',
       'Chicago', 'Chicago', 'New York', 'Dallas', 'New York', 'Dallas',
       'Dallas', 'Los Angeles', 'New York', 'Chicago', 'Los Angeles',
       'Los Angeles', 'Chicago', 'Dallas', 'Chicago', 'Philadelphia',
       'Dallas', 'Los Angeles', 'Dallas', 'Chicago', 'Los Angeles',
       'Philadelphia', 'Los Angel

In [318]:
print(X.shape)

print(y.shape)

encoder = LabelEncoder()
y_encoder = encoder.fit_transform(y) 

(500, 3)
(500,)


In [319]:
random_seed = int( random.random() * 100 )     ## 42

In [320]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoder, test_size=0.2, random_state=random_seed)

In [321]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(400, 3)
(100, 3)
(400,)
(100,)


In [322]:
y_train.dtype 

dtype('int32')

In [323]:
X_train = X_train.astype(  np.float32  )
X_test  = X_test.astype(   np.float32 )
y_train = y_train.astype(  np.int64  )
y_test  = y_test.astype(   np.int64 )

In [324]:
X_train_tr = torch.from_numpy(X_train)
X_test_tr  = torch.from_numpy(X_test)
y_train_tr = torch.from_numpy(y_train)
y_test_tr  = torch.from_numpy(y_test)

In [325]:
x_mean = X_train_tr.mean(0, keepdim=True)
x_std = X_train_tr.std(0, keepdim=True) + epsilon
X_train_tr = (X_train_tr - x_mean) / x_std
X_test_tr  = (X_test_tr - x_mean) / x_std

In [326]:
train_ds = TensorDataset( X_train_tr, y_train_tr  )

In [327]:
train_dl = DataLoader( train_ds, batch_size, shuffle=True  )

In [328]:
test_ds = TensorDataset(X_test_tr, y_test_tr)
test_dl = DataLoader(test_ds, batch_size=batch_size)


In [335]:
class MLP_Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 64),        # input: 3 features
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 128),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 5) 
        )
        
    def forward(self, x):
        return self.model(x)  


In [340]:
def training_loop( N_Epochs, model, loss_fn, opt  ):
    
    for epoch in range(N_Epochs):
        for xb, yb in train_dl:
            
            xb = xb.to( torch_device )
            yb = yb.to( torch_device )
            
            y_pred = model(xb)
            
            loss = loss_fn(y_pred, yb)
            
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        if epoch % 5 == 0:
            print(epoch, "loss=", loss)
        

In [341]:
model = MLP_Classifier()

opt     = torch.optim.Adam(    model.parameters(), lr=learning_rate )
loss_fn = nn.CrossEntropyLoss()

training_loop(  N_Epochs, model, loss_fn, opt  )

0 loss= tensor(1.6095, grad_fn=<NllLossBackward0>)
5 loss= tensor(1.5431, grad_fn=<NllLossBackward0>)
10 loss= tensor(1.5938, grad_fn=<NllLossBackward0>)
15 loss= tensor(1.5226, grad_fn=<NllLossBackward0>)
20 loss= tensor(1.5511, grad_fn=<NllLossBackward0>)
25 loss= tensor(1.6035, grad_fn=<NllLossBackward0>)
30 loss= tensor(1.5163, grad_fn=<NllLossBackward0>)
35 loss= tensor(1.5470, grad_fn=<NllLossBackward0>)
40 loss= tensor(1.4960, grad_fn=<NllLossBackward0>)
45 loss= tensor(1.6651, grad_fn=<NllLossBackward0>)
50 loss= tensor(1.4539, grad_fn=<NllLossBackward0>)
55 loss= tensor(1.6329, grad_fn=<NllLossBackward0>)
60 loss= tensor(1.4255, grad_fn=<NllLossBackward0>)
65 loss= tensor(1.5454, grad_fn=<NllLossBackward0>)
70 loss= tensor(1.5232, grad_fn=<NllLossBackward0>)
75 loss= tensor(1.7310, grad_fn=<NllLossBackward0>)
80 loss= tensor(1.4385, grad_fn=<NllLossBackward0>)
85 loss= tensor(1.5114, grad_fn=<NllLossBackward0>)
90 loss= tensor(1.3800, grad_fn=<NllLossBackward0>)
95 loss= tenso

In [342]:
def print_metrics_function(y_test, y_pred):
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
    print("Confusion Matrix:")
    print(confmat)
    print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred, average='weighted', zero_division =0))
    print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred, average='weighted', zero_division =0))
    print('F1-measure: %.3f' % f1_score(y_true=y_test, y_pred=y_pred, average='weighted', zero_division =0))

def testing_loop(test_dl, model, device):
    my_f1 = 0.0
    count = 0.0
    with torch.no_grad():
        for x_raw, y_raw in test_dl:
            x_true, y_true = x_raw.to(device), y_raw.to(device)
            _, y_pred = torch.max(model(x_true), dim=1)
            my_f1 += f1_score(y_true, y_pred, num_classes=10)
            count += 1
    return my_f1 / count

In [343]:
torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with torch.no_grad():
    for x_real, y_real in test_dl:
        
        batch_size = x_real.shape[0]
        
        ## x_real = x_real.view(  (batch_size, -1 ) )
        
        x_real = x_real.to( torch_device )
        
        y_pred = model(  x_real  )
        
        vals, indeces = torch.max( y_pred, dim=1  )
        preds = indeces
        print_metrics_function( y_real, preds.cpu() )

Accuracy: 0.28
Confusion Matrix:
[[2 4 1 0 0]
 [1 2 0 1 0]
 [0 2 2 1 4]
 [1 2 1 0 3]
 [0 0 1 1 3]]
Precision: 0.294
Recall: 0.281
F1-measure: 0.258
Accuracy: 0.34
Confusion Matrix:
[[3 1 2 0 1]
 [2 2 0 0 2]
 [0 0 2 1 1]
 [1 1 1 3 3]
 [1 1 1 2 1]]
Precision: 0.374
Recall: 0.344
F1-measure: 0.351
Accuracy: 0.19
Confusion Matrix:
[[0 0 3 2 2]
 [2 4 2 1 1]
 [2 2 0 0 2]
 [0 3 0 1 1]
 [1 2 0 0 1]]
Precision: 0.171
Recall: 0.188
F1-measure: 0.176
Accuracy: 0.25
Confusion Matrix:
[[1 0 1 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 1 0]]
Precision: 0.500
Recall: 0.250
F1-measure: 0.333


In [344]:
model.eval()

dummy_input = torch.randn(1, 3)

input_names  = ["input1"]
output_names = ["output1"]

torch.onnx.export(
        model, 
        dummy_input,
        "DLnet_WeatherData.onnx",
        verbose=False,
        input_names = input_names,
        output_names = output_names
)